In [1]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np

Load the Fashion MNIST dataset and split it into a training, a validation and a test set

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

Shuffle the training set

In [3]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=len(X_train) // 10)

2022-11-19 19:46:49.241278: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-19 19:46:49.241537: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-19 19:46:49.242591: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [4]:
for item_x, item_y in train_dataset.take(1):
    print(item_x, item_y)

tf.Tensor(
[[  0   0   0   0   1   0   1   1   4   0   0   0 139  89  87 141   7   0
    1   3   0   3   1   0   0   0   0   0]
 [  0   0   0   0   0   0   1   0   0   0  12 101 210 159 152 170  96  13
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   1   1   0   0  78 127 150 122 112 119 105  89 102 133
  112  56   0   0   1   0   0   0   0   0]
 [  0   0   0   0   0   1   0 118 156 115 102  90  84  82  82  79  79  82
  102 136  99   0   0   0   0   0   0   0]
 [  0   0   0   0   4   0  18 153 110 105 102  95  90  87  87  82  79  82
   82  90 133  15   0   1   0   0   0   0]
 [  0   0   0   0   4   0  56 153 129 124 101  95  89  89  86  84  84  79
   99 107 130  49   0   1   0   0   0   0]
 [  0   0   0   0   1   0  87 153 159 124  99  90  87  90  86  86  82  67
   90 136 122  72   0   3   0   0   0   0]
 [  0   0   0   0   0   0 110 144 199 127  99  96  95  95  95  93  89  69
  122 182 116  93   0   0   0   0   0   0]
 [  0   0   0   0   0   0 129 155 193 118 150 138 124

In [27]:
nx = item_x.numpy()
nx.reshape(28*28).tolist()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 41,
 150,
 77,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 71,
 56,
 117,
 9,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 66,
 132,
 68,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 90,
 88,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 103,
 195,
 117,
 194,
 185,
 54,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 31,
 246,
 201,
 207,
 197,
 203,
 229,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 62,
 226,
 185,
 191,
 192,
 195,
 225,
 45,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 178,
 220,
 194,
 197,
 194,
 206,
 244,
 159,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


Save each dataset to multiple TFRecord files

In [28]:
# we save to a single file instead of multiple. For multiple files implementation, check the official solution
with tf.io.TFRecordWriter("fashion_train_compressed.tfrecord", options=tf.io.TFRecordOptions(compression_type="GZIP")) as f:
    for item, label in train_dataset:
        image_feature = tf.train.Feature(int64_list=tf.train.Int64List(value=item.numpy().reshape(28*28).tolist())) # why don't we use float_list for the image? TODO try it
        label_feature = tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
        feature_dict = {"image": image_feature, "label": label_feature}
        features = tf.train.Features(feature=feature_dict)
        example = tf.train.Example(features=features)
        f.write(example.SerializeToString())


In [30]:
validation_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
with tf.io.TFRecordWriter("fashion_valid_compressed.tfrecord", options=tf.io.TFRecordOptions(compression_type="GZIP")) as f:
    for item, label in validation_dataset:
        image_feature = tf.train.Feature(int64_list=tf.train.Int64List(value=item.numpy().reshape(28*28).tolist()))
        label_feature = tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
        feature_dict = {"image": image_feature, "label": label_feature}
        features = tf.train.Features(feature=feature_dict)
        example = tf.train.Example(features=features)
        f.write(example.SerializeToString())

In [31]:
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
with tf.io.TFRecordWriter("fashion_test_compressed.tfrecord", options=tf.io.TFRecordOptions(compression_type="GZIP")) as f:
    for item, label in validation_dataset:
        image_feature = tf.train.Feature(int64_list=tf.train.Int64List(value=item.numpy().reshape(28*28).tolist()))
        label_feature = tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
        feature_dict = {"image": image_feature, "label": label_feature}
        features = tf.train.Features(feature=feature_dict)
        example = tf.train.Example(features=features)
        f.write(example.SerializeToString())

In [32]:
final_train = tf.data.TFRecordDataset(["fashion_train_compressed.tfrecord"], compression_type="GZIP")
final_valid = tf.data.TFRecordDataset(["fashion_valid_compressed.tfrecord"], compression_type="GZIP")
final_test = tf.data.TFRecordDataset(["fashion_test_compressed.tfrecord"], compression_type="GZIP")

Create a preprocessing layer

In [37]:
class Standardization(keras.layers.Layer):
    def adapt(self, data_sample):
        self.means_ = np.mean(data_sample, axis=0, keepdims=True)
        self.stds_ = np.std(data_sample, axis=0, keepdims=True)

    def call(self, inputs):
        return (inputs - self.means_) / (self.stds_ + keras.backend.epsilon())

In [36]:
feature_description = {
    "image": tf.io.FixedLenFeature([28*28], tf.int64),
    "label": tf.io.FixedLenFeature([], tf.int64)
}
for serialized_example in final_train.take(1):
    parsed_example = tf.io.parse_single_example(serialized_example, feature_description)

In [37]:
parsed_example

{'image': <tf.Tensor: shape=(784,), dtype=int64, numpy=
 array([  0,   0,   0,   0,   0,   0,   3,   0,   0,   0,  32, 115,  46,
         27,  29,  64, 141,  63,   0,   0,   0,   0,   3,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   6, 104, 173, 198,
        229, 252, 252, 252, 255, 247, 210, 215, 164,  77,   0,   0,   3,
          0,   0,   0,   0,   0,   0,   0,   0,   3,   0,  70, 199, 207,
        200, 189, 189, 197, 208, 214, 207, 190, 191, 195, 205, 210, 175,
         15,   0,   3,   0,   0,   0,   0,   0,   0,   0,   0,   8, 199,
        194, 188, 191, 191, 190, 182, 197, 198, 181, 189, 195, 191, 191,
        187, 211, 155,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
        103, 210, 191, 196, 197, 189, 192, 197, 194, 202, 192, 193, 194,
        189, 196, 194, 199, 208,  25,   0,   0,   0,   0,   0,   0,   0,
          0,   0, 151, 217, 196, 195, 195, 190, 194, 193, 197, 196, 196,
        197, 199, 195, 192, 200, 206, 213,  80,   0,   0,   0,   0, 